Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Come primo passo importo le librerie necessarie e i dati per un plot preliminare.

In [ ]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

In [ ]:
# Dizionario dei dati
dati = {}

# Costruisco i vettori
myList=['C2H2', 'CO2'] #modifica in percorsi relativi
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))


In [ ]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1],label=name)
    plt.legend()
    plt.grid()
